In [2]:
import random
import numpy as np
import math
import gym
from gym import Env
from gym.spaces import MultiDiscrete, Box
import torch
import torch.nn as nn
from transformers import GPT2Model, GPT2Config


In [ ]:

reward_list = []
GLOBAL_UAV1_PATH = []
GLOBAL_UAV2_PATH = []
pre_total_reward = 1000


In [1]:
import time

reward_list= []
GLOBAL_UAV1_PATH = []
GLOBAL_UAV2_PATH = []
pre_total_reward = 1000
class uavEnv(Env):
    def __init__(self):
        self.X_lau_location  = [950, 50]
        self.Y_lau_location  = [950, 50]
        self.UAV1_PATH = []
        self.UAV2_PATH = []
        self.UAV3_PATH = []
        self.UAV4_PATH = []
        self.start_time = time.time()
        self.g31_capacity = {}
        self.completed = 0
        self.account = 0 
        self.E = 0
        self.action_space = MultiDiscrete([5,5]) 
        self.observation_space = Box(low=np.array([-100]*4),high=np.array([1000]*4)) 
        self.state = np.array([10]*4)
        self.state[0] = self.X_lau_location[0]  
        self.state[1] = self.Y_lau_location[0]
        self.state[2] = self.X_lau_location[1]
        self.state[3] = self.Y_lau_location[1]     
        # number of users
        self.User_Location_X = [399, 91, 750, 956, 393, 790, 728, 211, 565, 971, 576, 266, 571, 401, 875, 837, 575, 585, 625, 788, 810, 792, 499, 140, 816, 391, 29, 36, 171, 530, 58, 546, 205, 89, 964, 117, 818, 64, 680, 981]
        self.User_Location_Y = [29, 590, 9, 448, 480, 633, 862, 588, 860, 103, 680, 86, 911, 948, 198, 933, 824, 270, 903, 825, 380, 641, 444, 725, 630, 150, 852, 382, 362, 463, 722, 621, 971, 420, 863, 853, 807, 317, 699, 991]

        Number_User = 40
        self.Max_flow_total = 0
        self.total_reward = 0

        
    def step(self, Action):

        M_LAU_length = 50
        Number_LAU = 2
        pently_lau = 0
        
    

        for i in range(Number_LAU):
            if Action[i] == 0:  #hover
                self.X_lau_location[i] = self.X_lau_location[i]
                self.Y_lau_location[i] = self.Y_lau_location[i]

            elif Action[i] == 1: # back
                self.Y_lau_location[i] = self.Y_lau_location[i] - M_LAU_length
                if self.Y_lau_location[i]<0: # Out of range
                    pently_lau = 1
                    self.Y_lau_location[i] = self.Y_lau_location[i] + M_LAU_length

            elif Action[i] == 2: # forward
                self.Y_lau_location[i] = self.Y_lau_location[i] + M_LAU_length
                if self.Y_lau_location[i]>1000:
                    pently_lau = 1
                    self.Y_lau_location[i] = self.Y_lau_location[i] - M_LAU_length

            elif Action[i] == 3: # left
                self.X_lau_location[i] = self.X_lau_location[i] - M_LAU_length
                if self.X_lau_location[i]<0:
                    pently_lau = 1
                    self.X_lau_location[i] = self.X_lau_location[i] + M_LAU_length

            elif Action[i] == 4: # right
                self.X_lau_location[i] = self.X_lau_location[i]+ M_LAU_length
                if self.X_lau_location[i]>1000:
                    pently_lau = 1
                    self.X_lau_location[i] = self.X_lau_location[i] - M_LAU_length

        Number_User =  len(self.User_Location_X)
        R_User_Location_X = []
        R_User_Location_Y = []
        Current_reward = 0
        for j in range(Number_User):
            for i in range(Number_LAU):
                D_GL = math.sqrt((self.User_Location_X[j]-self.X_lau_location[i])**2+(self.User_Location_Y[j]-self.Y_lau_location[i])**2)/1000
                if D_GL<0.1: # coverage of UAV :100m
                    self.completed = self.completed + 1
                    Current_reward = Current_reward + 10
                    R_User_Location_X.append(self.User_Location_X[j])
                    R_User_Location_Y.append(self.User_Location_Y[j])

            
          
        self.User_Location_X = [x for x in self.User_Location_X if x not in R_User_Location_X]     
        self.User_Location_Y = [x for x in self.User_Location_Y if x not in R_User_Location_Y]   
                    
                    
        
        self.account = self.account + 1
        self.state[0] = self.X_lau_location[0]  
        self.state[1] = self.Y_lau_location[0]
        self.state[2] = self.X_lau_location[1]
        self.state[3] = self.Y_lau_location[1]     
        U1 =[self.X_lau_location[0],self.Y_lau_location[0]]
        U2 =[self.X_lau_location[1],self.Y_lau_location[1]]
        self.UAV1_PATH.append(U1)
        self.UAV2_PATH.append(U2)

        
        
        reward =  Current_reward-(pently_lau)*100
        ##print(reward,self.account,self.completed)
        self.total_reward = self.total_reward + reward
        if self.account >= 200: # maximum step of UAVs
            done = True
            print(self.completed)
            reward_list.append(self.total_reward)
            info = {}
            return self.state, reward, done, info
        
        else:
            done = False
            
        global pre_total_reward
        global GLOBAL_UAV1_PATH
        global GLOBAL_UAV2_PATH
        if self.completed >= 40:
            done = True
            reward = reward+ 20*(100-self.account)
            self.total_reward = self.total_reward + reward
            if self.account < pre_total_reward :
                pre_total_reward = self.account
                # trajectory of UAVs
                GLOBAL_UAV1_PATH = self.UAV1_PATH
                GLOBAL_UAV2_PATH = self.UAV2_PATH
            reward_list.append(self.total_reward)
            info = {}
            print(self.account,self.total_reward)
            return self.state, reward, done, info
        else:
            done = False
        

        info = {}
        return self.state, reward, done, info

    def render(self):
        pass 
    
    
    def reset(self):
        self.account = 0
        self.completed = 0
        self.total_reward = 0
        self.Max_flow_total = 0
        self.average_delay = 0
        self.X_lau_location = []
        self.Y_lau_location = []
        # reset the initial locations of UAVs
        for _ in range(2):
            self.X_lau_location.append(random.randint(0, 1000))
            self.Y_lau_location.append(random.randint(0, 1000))
            
        self.g31_capacity = {}
        self.UAV1_PATH = []
        self.UAV2_PATH = []
        self.UAV3_PATH = []
        self.UAV4_PATH = []
        Number_User = 40
        self.User_Location_X = [399, 91, 750, 956, 393, 790, 728, 211, 565, 971, 576, 266, 571, 401, 875, 837, 575, 585, 625, 788, 810, 792, 499, 140, 816, 391, 29, 36, 171, 530, 58, 546, 205, 89, 964, 117, 818, 64, 680, 981]
        self.User_Location_Y = [29, 590, 9, 448, 480, 633, 862, 588, 860, 103, 680, 86, 911, 948, 198, 933, 824, 270, 903, 825, 380, 641, 444, 725, 630, 150, 852, 382, 362, 463, 722, 621, 971, 420, 863, 853, 807, 317, 699, 991]
        self.E = 0
        self.E_CC = 0
        self.state[0] = self.X_lau_location[0]  
        self.state[1] = self.Y_lau_location[0]
        self.state[2] = self.X_lau_location[1]
        self.state[3] = self.Y_lau_location[1]     
        return self.state
    
env = uavEnv()

def collect_trajectories(env, num_trajectories):
    trajectories = []
    for _ in range(num_trajectories):
        state = env.reset()
        trajectory = []
        done = False
        while not done:
            action = env.action_space.sample()
            next_state, reward, done, _ = env.step(action)
            trajectory.append((state, action, reward, next_state, done))
            state = next_state
        trajectories.append(trajectory)
    return trajectories

trajectories = collect_trajectories(env, 1000)


NameError: name 'Env' is not defined

In [ ]:
#Define the Decision Transformer Model
import torch
import torch.nn as nn
from transformers import GPT2Model, GPT2Config

class DecisionTransformer(nn.Module):
    def __init__(self, state_dim, action_dim, max_length=50):
        super(DecisionTransformer, self).__init__()
        config = GPT2Config(vocab_size=1, n_positions=max_length, n_embd=128, n_layer=4, n_head=8)
        self.transformer = GPT2Model(config)
        self.state_emb = nn.Linear(state_dim, config.n_embd)
        self.action_emb = nn.Linear(action_dim, config.n_embd)
        self.reward_emb = nn.Linear(1, config.n_embd)
        self.predict_action = nn.Linear(config.n_embd, action_dim)
        
    def forward(self, states, actions, rewards):
        state_embeddings = self.state_emb(states)
        action_embeddings = self.action_emb(actions)
        reward_embeddings = self.reward_emb(rewards.unsqueeze(-1))
        
        inputs = state_embeddings + action_embeddings + reward_embeddings
        outputs = self.transformer(inputs)
        predicted_actions = self.predict_action(outputs)
        return predicted_actions


In [ ]:
#Training The Decision Transformer
def preprocess_trajectories(trajectories):
    states, actions, rewards = [], [], []
    for traj in trajectories:
        states.append(np.array([step[0] for step in traj]))
        actions.append(np.array([step[1] for step in traj]))
        rewards.append(np.array([step[2] for step in traj]))
    return np.array(states), np.array(actions), np.array(rewards)

states, actions, rewards = preprocess_trajectories(trajectories)

model = DecisionTransformer(state_dim=4, action_dim=2)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
loss_fn = nn.MSELoss()

for epoch in range(1000):  # Number of epochs
    for i in range(len(states)):
        state_batch = torch.tensor(states[i], dtype=torch.float32)
        action_batch = torch.tensor(actions[i], dtype=torch.float32)
        reward_batch = torch.tensor(rewards[i], dtype=torch.float32)

        optimizer.zero_grad()
        predicted_actions = model(state_batch, action_batch, reward_batch)
        loss = loss_fn(predicted_actions, action_batch)
        loss.backward()
        optimizer.step()
        
    print(f'Epoch {epoch}, Loss: {loss.item()}')


In [ ]:
#Evaluate The Model
def evaluate_model(env, model):
    state = env.reset()
    done = False
    total_reward = 0

    while not done:
        state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
        action = model(state_tensor, torch.zeros_like(state_tensor), torch.zeros(1, 1)).argmax(dim=-1).item()
        next_state, reward, done, _ = env.step([action])
        total_reward += reward
        state = next_state

    return total_reward

reward = evaluate_model(env, model)
print(f'Total Reward: {reward}')
